In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import re
import os
%matplotlib inline

# Analysis options

We have to make the exact same analysis but with some different parameters. The following parameters change the outputs.

In [ ]:
# OBSOLETE OPTION -- Leave it set to "aa".
# Which downsampling normalisation to use: This selects input data:
# dd: Deduplicated downsampled data -- This is not particularly interesting
# aa: Analyse all reads in data, downsample based on number of PF reads
DS_MODE = "aa"

In [ ]:
# This option controls whether the Swift long-insert libraries are included in the analysis.
# They were sequenced on the HiSeq 4000 with less coverage, and are thus not directly comparable.

INCLUDE_SWIFT_LONG_LIBS = False

# Alignment notes

Alignment with bwa-mem.

The minimum seed length is 19.

Maximum gap length: 100.

## Load Picard data

### Step 1 is to load all the metrics from the downsampled data.

In [ ]:
def load_picard(path):
    with open(path) as f:
        mode = None
        metrics_header = None
        metrics_data = {}
        histogram_series = []
        histogram_data = []
        for l in f:
            line = l.rstrip("\r\n")
            if mode == 'metrics':
                metrics_header = line.split()
                mode = 'metrics2'
            elif mode == 'metrics2':
                if line != "":
                    metrics_data = dict(zip(metrics_header, line.split()))
                else:
                    mode = None
            elif mode == 'histogram':
                histogram_series = line.split()
                histogram_data = [list() for _ in histogram_series]
                mode = 'histogram2'
            elif mode == 'histogram2':
                if line != "":
                    for i, value in enumerate(line.split()):
                        histogram_data[i].append(value)
                else:
                    mode = None
            elif line.startswith("## METRICS CLASS"):
                mode = 'metrics'
            elif line.startswith("## HISTOGRAM"):
                mode = 'histogram'
        if histogram_series:
            #metrics_data['histograms'] = dict(zip(histogram_series, histogram_data))
            for name, data in zip(histogram_series, histogram_data):
                metrics_data[name] = [float(x) for x in data]
    return metrics_data

In [ ]:
samples = []
for bam in glob("30_downsample/*-{}_DS_MD.AlignmentSummaryMetrics.txt".format(DS_MODE)):
    basepath = re.sub(r"_MD\.AlignmentSummaryMetrics\.txt$", "", bam)
    data = {}
    try:
        for metrics in ['_MD.AlignmentSummaryMetrics', '_MD.InsertSizeMetrics',
                        '.MarkDuplicatesMetrics', '_MD.WgsMetrics']:
            new_data = load_picard("{}{}.txt".format(basepath, metrics))
            if any(k in data for k in new_data):
                print("Duplicate key {} found in {}".format(k, metrics))
            data.update(new_data)
    except FileNotFoundError as e:
        print("File {} not found, skipping this sample.".format(e.filename))
        continue
    samples.append(data)

In [ ]:
print(len(samples))

### Step 2: Get original number of PF reads from before downsampling.

In [ ]:
inital_read_counts = []
for alignment_txt in glob("20_piccard/*.AlignmentSummaryMetrics.txt"):
    library = re.match(r"([^/]+)\.AlignmentSummaryMetrics.txt", os.path.basename(alignment_txt)).group(1)
    new_data = load_picard(alignment_txt)
    inital_read_counts.append({'LIBRARY': library, 'INITIAL_READS': int(new_data['TOTAL_READS']),
                                    'INITIAL_READS_PER_END': int(new_data['TOTAL_READS']) / 2,
                                    'INITIAL_READS_ALIGNED': int(new_data['PF_READS_ALIGNED']),
                                    'INITIAL_READS_ALIGNED_PER_END': int(new_data['PF_READS_ALIGNED']) / 2})

In [ ]:
dedup_read_counts = []
for rc_txt in glob("20_piccard_dd/*.readCount.txt"):
    library = re.match(r"([^/]+)\.readCount\.txt", os.path.basename(rc_txt)).group(1)
    rc = int(open(rc_txt).read().strip())
    dedup_read_counts.append({'LIBRARY': library, 'DEDUPLICATED_READS': rc, 'DEDUPLICATED_READS_PER_END': rc/2})

### Step 3: Get arabidopsis metrics.

In [ ]:
arabidopsis_samples = []
for asm in glob("arabidopsis/20_piccard/*.AlignmentSummaryMetrics.txt"):
    basepath = re.sub(r"\.AlignmentSummaryMetrics\.txt$", "", asm)
    data = {}
    for metrics in ['.AlignmentSummaryMetrics', '.InsertSizeMetrics',
                    '.MarkDuplicatesMetrics', '.WgsMetrics']:
        new_data = load_picard("{}{}.txt".format(basepath, metrics))
        if any(k in data for k in new_data):
            print("Duplicate key {} found in {}".format(k, metrics))
        data.update(new_data)
    arabidopsis_samples.append(data)

In [ ]:
if arabidopsis_samples:
    print("We have arabidopsis data")
else:
    print("We don't have arabidopsis data, but will continue anyway")

In [ ]:
main_df = pd.DataFrame(samples).reindex().apply(pd.to_numeric, axis=0, errors='ignore')
in_reads_df = pd.DataFrame(inital_read_counts)
dd_reads_df = pd.DataFrame(dedup_read_counts)
arabidopsis_df = pd.DataFrame(arabidopsis_samples).apply(pd.to_numeric, axis=0, errors='ignore')


# Without Arabidopsis data
if arabidopsis_df.empty:
    df = pd.merge( 
        pd.merge(
            main_df, 
            in_reads_df,  
            on='LIBRARY').reindex(), 
        dd_reads_df,  
        on='LIBRARY')
else: # With Arabidopsis
    df = pd.merge( 
        pd.merge( 
            pd.merge( 
                main_df, 
                arabidopsis_df, 
                on='LIBRARY', suffixes=('', '_AR')).reindex(), 
            in_reads_df,  
            on='LIBRARY').reindex(), 
        dd_reads_df,  
        on='LIBRARY')
    df.dtypes[0:10]

In [ ]:
df['Kit'] = pd.Categorical(df.LIBRARY.str.split("-").str.get(0))
df['Conc'] = pd.Categorical(df.LIBRARY.str.split("-").str.get(1), categories=['10ng', '100ng'])

Optionally remove `Swiftlong`, and fix the order of kits and concentrations.

In [ ]:
if not INCLUDE_SWIFT_LONG_LIBS:
    df = df.loc[df.Kit != "Swiftlong"]
    df.Kit = df.Kit.cat.set_categories(['Kapa', 'NEB', 'Nextera', 'Quanta', 'Swift2S'])
else:
    df.Kit = df.Kit.cat.set_categories(['Kapa', 'NEB', 'Nextera', 'Quanta', 'Swift2S', 'Swiftlong'])

In [ ]:
df_100 = df.loc[df.Conc == "100ng"]
df.head(n=1)

In [ ]:
print("Number of entries: ", len(df))

# Utility function

In [ ]:
def savefig(name, main=True):
    for fmt in ['png', 'pdf']:
        if main:
            if INCLUDE_SWIFT_LONG_LIBS:
                plt.savefig('plots-main-withLong/{}.{}'.format(name, fmt))
            else:
                plt.savefig('plots-main/{}.{}'.format(name, fmt))
        else:
            if INCLUDE_SWIFT_LONG_LIBS:
                plt.savefig('plots-suppl-withLong/{}.{}'.format(name, fmt))
            else:
                plt.savefig('plots-suppl/{}.{}'.format(name, fmt))

# Number of reads

Number of PE reads (reads per end):

In [ ]:
DOWNSAMPLING_TARGET_READS = 90e6

df['TOTAL_READS_PER_END'] = df['TOTAL_READS'] / 2
num_reads_downsampled = df.TOTAL_READS_PER_END.mean()
num_reads_initial = df.INITIAL_READS_PER_END.mean()
num_reads_deduplicated = df.DEDUPLICATED_READS_PER_END.mean()
print("One lane of HiSeqX:       {:9d} (optimistic)".format(400000000))
print("Initial reads (avg):      {:9d}".format(int(num_reads_initial)))
print("Deduplicated reads (avg): {:9d}".format(int(num_reads_deduplicated)))
print("Downsampled reads:        {:9d}".format(int(num_reads_downsampled)))
print("Target downsampled reads  {:9d}".format(int(DOWNSAMPLING_TARGET_READS)))

Experiment set-up:

One pool for each kit. There are 8 libraries in each pool. The pool is sequenced on four lanes. Each library gets a half lane's worth of reads on average.

The mean read count per library is around half of the expected 200M, because of less than optimal yield, and inevitable differences between samples in the pool.

The "downsampled" in the table above depends on the constant set at the top of this notebook. It is either based on the lowest Initial reads (aa) or the lowest Deduplicated reads (dd).

In [ ]:
print(DS_MODE)

Downsampling is based on the following read counts: aa=INITIAL, dd=DEDUPLICATED.

The point with the fewest reads represents the value used for downsampling (after rounding down).

In [ ]:
sns.boxplot(data=df, x='Kit', y='INITIAL_READS_PER_END', hue='Conc')
plt.ylabel('Number of reads per replicate')
plt.xlabel("")
savefig("total_number_of_reads")
plt.show()
sns.boxplot(data=df, x='Kit', y='DEDUPLICATED_READS_PER_END', hue='Conc')
plt.title('Deduplicated reads')
plt.show()
sns.boxplot(data=df, x='Kit', y='TOTAL_READS_PER_END', hue='Conc')
plt.title('Downsampled reads')

In [ ]:
print(df[(df.Kit=='Swiftlong') & (df.Conc == '100ng')].PF_READS/2)
print((df[(df.Kit=='Swiftlong') & (df.Conc == '100ng')].PF_READS/2).mean())

# Aligned reads

In [ ]:
print([col for col in list(df) if 'ALIGNED' in col.upper()])

## Plot of any aligned reads

In [ ]:
sns.boxplot(data=df, x='Kit', y='PCT_PF_READS_ALIGNED', hue='Conc')
plt.title('Reads aligned to human genome')
plt.ylabel("Fraction of reads")
plt.xlabel("")

## Plots for MAPQ > 20 reads

Requiring MAPQ > 20 removes a large percentage of reads. We found a lot of reads that map to both references.

In [ ]:
df['PCT_HQ_PF_READS_ALIGNED'] = df.PF_HQ_ALIGNED_READS / df.TOTAL_READS
sns.boxplot(data=df, x='Kit', y='PCT_HQ_PF_READS_ALIGNED', hue='Conc')
plt.title('Reads aligned to human genome (MAPQ>=20)')
plt.ylabel("Fraction of reads")
plt.xlabel("")
savefig("aligned_reads_human")

In [ ]:
print("Low aligned rate")
df[df.PCT_HQ_PF_READS_ALIGNED < 0.88][["LIBRARY", "PCT_HQ_PF_READS_ALIGNED"]]

In [ ]:
# Correct the aligned pct. All reads in the AR dataset are aligned.
if not arabidopsis_df.empty:
    df['PCT_HQ_PF_READS_ALIGNED_AR'] = df.PF_HQ_ALIGNED_READS_AR / df.INITIAL_READS
    sns.boxplot(data=df, x='Kit', y='PCT_HQ_PF_READS_ALIGNED_AR', hue='Conc')
    plt.title('Reads aligned to Arabidopsis Lyrata genome (MAPQ >= 20)')
    plt.ylabel("Fraction of reads")
    plt.xlabel("")
    savefig("aligned_reads_arabidopsis")

## Show arabidopsis in same plot

In [ ]:
if not arabidopsis_df.empty:
    grid = sns.boxplot(data=df, x='Kit', y='PCT_HQ_PF_READS_ALIGNED', hue='Conc', linewidth=0.5, fliersize=3)
    grid.legend().remove()
    sns.boxplot(data=df, x='Kit', y='PCT_HQ_PF_READS_ALIGNED_AR', hue='Conc', linewidth=0.5, fliersize=3)
    plt.ylim(0,1)
    savefig("aligned_reads_both", False)

Alternative way to do combined plot here -- not used:

In [ ]:
if not arabidopsis_df.empty:
    #df['PCT_HQ_PF_READS_ALIGNED_AR'] = df.PF_HQ_ALIGNED_READS_AR / df.INITIAL_READS
    #df['PCT_HQ_PF_READS_ALIGNED'] = df.PF_HQ_ALIGNED_READS / df.TOTAL_READS
    hdf = pd.DataFrame(df)
    hdf['Value'] = hdf.PCT_HQ_PF_READS_ALIGNED
    hdf['Species'] = "Human"
    adf = pd.DataFrame(df)
    adf['Value'] = adf.PCT_HQ_PF_READS_ALIGNED_AR
    adf['Species'] = "Arabidopsis"
    combined = pd.concat([adf, hdf])

Show details for one sample. This is to determine why there is so many reads aligned to both arabidopsis and human at the same time. This was never completely solved, but we see that the problem goes away when requiring a moderate mapping quality (20).

In [ ]:
if not arabidopsis_df.empty:
    sample = df[df['LIBRARY'] == 'NEB-10ng-1'].iloc[0]
    print("sample:", sample.LIBRARY)
    print("initial read:", sample.INITIAL_READS)
    print("initial (pre-DS) reads aligned to human genome: {} ({:.1f} %)".format(
          sample.INITIAL_READS_ALIGNED, (sample.INITIAL_READS_ALIGNED * 100 / sample.INITIAL_READS)))
    print("reads aligned to arabidopsis genome: {} ({:.1f} %)".format(
            sample.PF_READS_ALIGNED_AR, (sample.PF_READS_ALIGNED_AR * 100 / sample.INITIAL_READS)) )
    FLAG_2_NREADS_NEB_10ng_1 = 89977265# Filtered on SAM flag 2
    assert sample.LIBRARY == "NEB-10ng-1"
    print("reads aligned PROPER PAIR to arabidopsis: {} ({:.1f} %)".format(
            FLAG_2_NREADS_NEB_10ng_1, (FLAG_2_NREADS_NEB_10ng_1 * 100 /sample.INITIAL_READS)) )

### Mapping quality technical definition

$$
MQ = -\log_{10} \left( 1.0 - \frac{10^{-SUM\_BASE\_Q(best)}}{\sum_i 10^{-SUM\_BASE\_Q(i)}} \right)
$$



The mapping quality definition is defined based on the sum of base qualities of **mismatching bases**, as compared to the reference.

The fraction above compares the mismatching base quality at the best hit to the mismatching base qualities at the other possible hits.

### Mapping quality description

The mapping quality is defined as a Phred-scaled probability that the mapped position of the read is the single best location in the genome. The probability is computed based on the sum of Q-scores of bases in the read that do not match the reference. A high mapping quality signifies that the read is uniquely mapped and a good match to the reference. Most reads will have a mapping quality of 60, indicating this condition. A mapping quality of 20 indicates a probability of 99 % that the read is mapped uniquely and correctly. Multi-mapping reads will in general have mapping qualities close to zero.

### Why is the KAPA kit so bad at producing aligned reads?

The KAPA kit scores poorly on alignment, with a low percentage aligned, both using MAPQ cut-off or not.

KAPA has a very broad insert size distribution, almost not peaking. We can compare with Swift, which has such a distribution for 100 ng.

In [ ]:
pd.DataFrame({
    "Kit": ["Kapa", "Kapa", "Swift", "Swift"],
    "Conc":["10ng", "100ng","10ng",  "100ng"],
    "Broad distribution": [True, True, False, True],
    "Bad align frac": [True, False, False, True],
    })

The broad size distribution and bad alignment don't seem directly correlated.

# Insert size (Picard)

In [ ]:
colours = sns.color_palette()
kits = df.Kit.cat.categories
KIT_COL = dict(zip(kits, colours))
print(KIT_COL)

## Simple plot with read count

In [ ]:
mean_insert_sizes = []
for conc in ["10ng", "100ng"]:
    legends = set()
    plt.figure()
    plt.xlim(0, 900)
    plt.title(conc)
    for kit, group in df[df.Conc==conc].groupby(("Kit"), as_index=False):
        colour = KIT_COL[kit]
        means = []
        for ix, (xs, ys) in group[['insert_size', 'All_Reads.fr_count']].iterrows():
            if not kit in legends:
                plt.plot(xs, ys, color=colour, label=kit)
                legends.add(kit)
            else:
                plt.plot(xs, ys, color=colour)
            # Mean
            values = np.array(xs)
            weights = np.array(ys)
            mean = sum(weights*values) / sum(weights)
            means.append(mean)
        mean_insert_sizes.append((conc, kit, sum(means) / len(means)))
            
    plt.legend()

### Plot with range

In [ ]:
for conc in ["10ng", "100ng"]:
    legends = set()
    plt.figure()
    plt.xlim(30, 900)
    plt.title(conc)
    for kit, group in df[df.Conc==conc].groupby(("Kit"), as_index=False):
        colour = KIT_COL[kit]
        means = []
        for ix, (xs, ys) in group[['insert_size', 'All_Reads.fr_count']].iterrows():
            x_sel = xs[30:]
            y_sel = np.array(ys[30:])
            y_sel = y_sel / 1e6 # y_sel.sum()
            if not kit in legends:
                plt.plot(x_sel, y_sel, color=colour, label=kit)
                legends.add(kit)
            else:
                plt.plot(x_sel, y_sel, color=colour)
    plt.legend()
    plt.xlabel("Insert size")
    plt.ylabel("Frequency [millions of reads]")
    savefig("insert_size_{}".format(conc), False)

In [ ]:
for vvv in mean_insert_sizes:
    print(*vvv, sep="\t")

In [ ]:
nextera10_samples = df[(df.Conc=="10ng") & (df.Kit=="Nextera")]
plt.figure()
plt.xlim(100, 130)
plt.ylim(0, 240000)
plt.title("Nextera 10ng Zoom Length 100-130")
nextera10_samples[['insert_size', 'All_Reads.fr_count']].apply(
            lambda x: plt.plot(*x.apply(lambda x: x[100:130]), '.'), 
    axis=1)
plt.grid()
savefig("nextera_insert_size_100", False)
plt.figure()
plt.xlim(200, 230)
plt.ylim(0, 240000)
plt.title("Nextera 10ng Zoom Length 200-230")
nextera10_samples[['insert_size', 'All_Reads.fr_count']].apply(
            lambda x: plt.plot(*x.apply(lambda x: x[200:230]), '.'), 
    axis=1)
plt.grid()
savefig("nextera_insert_size_200", False)

The period of the oscillations is approximately 10 bases. The magnitude is approximately 10000 reads, similar in both size ranges, less fraction of the reads at 200.

## Short fragments

In [ ]:
SHORT_FRAG = 28
for i, row in df_100.iterrows():
    col = KIT_COL[row.Kit]
    plt.plot(row['insert_size'], row['All_Reads.fr_count'], color=col)
plt.plot(
    [df['insert_size'][0][SHORT_FRAG], df['insert_size'][0][SHORT_FRAG]],
    [0, max(df['All_Reads.fr_count'][0][0:SHORT_FRAG])],
    'r'
    )
plt.xlim(0, 100)
plt.xlabel("Insert size")
plt.ylabel("Number of reads")
savefig("short_insert_zoom_100ng", False)

In [ ]:
df['short_fragments_pct'] = df['All_Reads.fr_count'].map(lambda x: sum(x[0:SHORT_FRAG]) * 100.0 / sum(x))

In [ ]:
sns.boxplot(data=df, x='Kit', y='short_fragments_pct', hue='Conc', linewidth=0.5, fliersize=3)
plt.ylim(0, 4)
plt.ylabel("Percentage of short fragments")
plt.xlabel("")
savefig("short_fragments_fraction", False)

The peak at short fragment lengths looks serious, but only accounts for max. 4 % of the reads. NEB has more such reads.

## Insert size related summary tables

In [ ]:
def get_mean(row):
    values = row['insert_size']
    freqs = row['All_Reads.fr_count']
    mean = sum(v*f for v, f in zip(values, freqs)) / sum(freqs)

def get_median(row):
    values = row['insert_size']
    freqs = row['All_Reads.fr_count']
    midpoint = sum(freqs) / 2
    acc = 0
    loc = 0
    while loc < len(freqs):
        acc += freqs[loc]
        if acc > midpoint:
            return values[loc]
        loc += 1
    return values[-1]



In [ ]:
insert_size = df.groupby(["Conc", "Kit"])[['short_fragments_pct', 'MEDIAN_INSERT_SIZE', 'MEAN_INSERT_SIZE', 'MEDIAN_ABSOLUTE_DEVIATION', 'STANDARD_DEVIATION']].mean()
insert_size.to_csv("result-tables/insert-size.csv")
insert_size

### TODO.maybe: reproducibility of insert size between replicates

# Duplicated reads

The downsampling significantly distorted the duplication rates.

The old code and plots for duplicates were junk and have been deleted.

See the notebook in subprojects/Undownsampled.ipynb

# Genome coverage

Reference scale for coverage: HiSeqX single lane sequencing.

The PF reads spec for HiSeqX is 2.6 to 3.0 billion per flow cell, so 2.6 B should also provide a 30X genome.

In [ ]:
print("*All quantities below in Million reads*")
reads_for_30x = 2.6 * 1000 / 8
print("PF reads per lane for 30X coverage, per end:", reads_for_30x)
expected_per_library_reads = (reads_for_30x * 4) / 8
print("Expected reads per library, 8 libraries over 4 lanes:", expected_per_library_reads)
actual_reads = df.TOTAL_READS.mean() // (2*1000000) # Divide by two, as TOTAL_READS counts both PE ends
print("Actual downsampled read count:", actual_reads)
expected_coverage_level = 30 * (actual_reads / reads_for_30x)
print("")
print("Expected coverage level based on read count:", expected_coverage_level)
print("")

In [ ]:
plot_max_cov = 16

In [ ]:
test_sample = df.loc[1]

In [ ]:
sns.distplot(test_sample['coverage'], kde=False,
             hist_kws={"weights":test_sample['high_quality_coverage_count']})

In [ ]:
plt.plot(test_sample['coverage'], test_sample['high_quality_coverage_count'])
plt.xlim(0, plot_max_cov)

In [ ]:

for conc in ["10ng", "100ng"]:
    legends = set()
    plt.xlim(0, plot_max_cov)
    plt.title(conc)
    plt.ylabel("Number of genomic loci")
    plt.xlabel("Coverage (number of covering reads)")

    for kit, group in df[df.Conc==conc].groupby("Kit", as_index=False):
        colour = KIT_COL[kit]
        for ix, (xs, ys) in group[['coverage', 'high_quality_coverage_count']].iterrows():
            if not kit in legends:
                plt.plot(xs, ys, color=colour, label=kit)
            else:
                plt.plot(xs, ys, color=colour)
            legends.add(kit)
    plt.legend()
    savefig("/coverage_{}".format(conc), False)
    plt.show()

Number of entries, reads

In [ ]:
for ix, sample in df.iterrows():
    hist_sum = sum(sample['high_quality_coverage_count'])
    hist_weighted_sum = sum(c * n for c, n in zip(sample['coverage'], 
                                                  sample['high_quality_coverage_count']))
    print(sample['LIBRARY'], "\t:\t", hist_sum, "\t / ", hist_weighted_sum)

NUM_GENOMIC_LOC = hist_sum

# Cumulative coverage

In [ ]:
df['cumulative_num_reads'] = df.high_quality_coverage_count.apply(np.cumsum)
df['cumulative_frac_reads'] = df.cumulative_num_reads.apply(lambda x: x / NUM_GENOMIC_LOC)

In [ ]:
df.cumulative_num_reads.loc[3][-1] # Number of genomic loci

In [ ]:
legends = set()

fig = plt.figure(figsize=(10, 20))
axs = {
    "10ng": fig.add_subplot(2, 1, 1),
    "100ng": fig.add_subplot(2, 1, 2)
}
for conc, ax in axs.items():
    ax.set_xlim(0, plot_max_cov)
    ax.set_title(conc)
    ax.set_ylabel("Fraction of genome")
    ax.set_xlabel("Coverage")
for (kit, conc), group in df.groupby(("Kit", "Conc"), as_index=False):
    colour = KIT_COL[kit]
    ax = axs[conc]
    for ix, (xs, yys) in group[['coverage', 'cumulative_frac_reads']].iterrows():
        ys = 1.0 - yys
        if not (ax, kit) in legends:
            ax.plot(xs, ys, color=colour, label=kit)
        else:
            ax.plot(xs, ys, color=colour)
        legends.add((ax, kit))
for ax in axs.values():
    ax.legend()

In [ ]:
fig = plt.figure(figsize=(10, 10))


for kit, group in df[df.Conc=='100ng'].groupby("Kit", as_index=False):
    ys = 1.0 - np.mean(group['cumulative_frac_reads'])
    xs = group['coverage'].iloc[0]
    plt.plot(xs, ys, color=KIT_COL[kit], label=kit)

plt.legend()
plt.plot([expected_coverage_level, expected_coverage_level], [0.0,1.0], 'c--')
plt.xlim(0, plot_max_cov)
plt.ylim(0, 1)
plt.title("averaged over replicates -- 100 ng")
plt.ylabel("Fraction of genome covered at or above")
plt.xlabel("Coverage")


## Mean coverage results

In [ ]:
mean_coverage_summary = df.groupby(['Conc','Kit'], as_index=False)[
    ['MEAN_COVERAGE','MEDIAN_COVERAGE', 'PCT_1X',
     'PCT_10X', 'TOTAL_READS', 'INITIAL_READS']
    ].mean()
mean_coverage_summary.to_csv("result-tables/mean-coverage.csv")
mean_coverage_summary

In [ ]:
sns.boxplot(data=df, x="Kit", y="MEAN_COVERAGE", hue="Conc")
plt.ylim(0, 8)
plt.ylabel('Mean coverage per {:.0f}M read pairs'.format(DOWNSAMPLING_TARGET_READS/1e6))
plt.xlabel("")
savefig("mean_coverage")

## Old analysis

The analysis was first done with a different reference, including ALTernative contigs. It's probably not a good idea to use these ALT contigs unless the whole pipeline takes them into account all the time.

In [ ]:
result_old_ref = """Kit,MEAN_COVERAGE,MEDIAN_COVERAGE,PCT_1X,PCT_10X,TOTAL_READS
Kapa,4.84100525,5.0,0.90678325,0.044468,179998242.0
NEB,4.42826475,4.25,0.9013639999999999,0.0293165,180011693.5
Nextera,6.6390705,7.0,0.9147957499999999,0.1851845,179995698.5
Quanta,4.7782167499999995,5.0,0.9069119999999999,0.041523500000000005,179997947.5
Swift2S,4.76407625,4.75,0.9061827499999999,0.041947250000000005,179996364.0"""
pd.DataFrame((c.split(',') for c in result_old_ref.splitlines()))

In [ ]:
#table.to_csv("")

In [ ]:
print(df['coverage'][0][0])
print(df['high_quality_coverage_count'][0][0])

Note on interpretation of coverage histogram: 

In [ ]:
# Aligned reads more detailed, todo
read_counts = []

for i, row in df.iterrows():
    library = {
        'LIBRARY': row.LIBRARY,
        'Kit': row.Kit,
        'Conc': row.Conc
    }
    read_counts.append(dict(
                        Reads=row.TOTAL_READS_PER_END,
                        Type='Downsampled',
                        **library))
    aligned = 1

# Coverage calculation detail: Counting of overlapping reads